In [1]:
import pdftotext
import pandas as pd

In [2]:
#path_in = '/home/philipp/Data/obf_autman/pdf/Manuale_Johannklause_20201109.pdf'
path_in = '/home/philipp/Data/obf_autman/pdf/Manuale_Dross.pdf'

In [3]:
def other_tool(filename, end):
    data = []
    with open(filename, "rb") as f:
        pdf = pdftotext.PDF(f)  
    for i, page in enumerate(pdf) :
        data.append({'page': i , 'content': page})
        if i == end:
            break
    return data

In [4]:
#path_in = '/home/philipp/Data/obf_autman/pdf/20201203_PD_Manuale_Pernitz_test.pdf'

In [5]:
pdf_data = other_tool(path_in, 100)

In [6]:
pdf_data

[{'page': 0,
  'content': ' FB 172\n FR 02\n          Forstbetrieb Waldviertel Voralpen\n          Droß\n                                                                        Druckdatum: 31.03.2021                   BKL         Umtrieb               Wirtschaftswald\n                                                                                                                                                                                 053 A 1\n TO: 1420 Laufzeit: 01.01.2022- 31.12.2031\n                                                                         Seite 1 von 882                       9010            140                 in Ertrag\n                                                                                                                                                                                            3,06 ha\n    Seeh [m]              Exp             Neig [%]             Neig [°]             STOE                 VTYP             Wald-Typ          Wuchs-

In [44]:
txt = pdf_data[7]['content']

In [45]:
'''for typ in ['#BZ', '#T', '#N', '#G']:
    start = txt.find(typ)
    print(start)
    end = txt[start+1:].find('#') + start + 1
    print(txt[start:end])'''

"for typ in ['#BZ', '#T', '#N', '#G']:\n    start = txt.find(typ)\n    print(start)\n    end = txt[start+1:].find('#') + start + 1\n    print(txt[start:end])"

In [46]:
txt

' FB 172\n FR 02\n          Forstbetrieb Waldviertel Voralpen\n          Droß\n                                                              Druckdatum: 31.03.2021                   BKL         Umtrieb              Wirtschaftswald\n                                                                                                                                                                      053 C 2\n TO: 1420 Laufzeit: 01.01.2022- 31.12.2031\n                                                               Seite 8 von 882                       9010            100                in Ertrag\n                                                                                                                                                                                 2,36 ha\n    Seeh [m]          Exp           Neig [%]         Neig [°]             STOE                VTYP             Wald-Typ          Wuchs-Geb. Verbissgrad              Schälgrad       fr.SS\n      500             NO    

In [47]:
x = txt[txt.find('Wirtschaftswald\n')+20:txt.find('\n TO')].strip()

In [48]:
x

'053 C 2'

## Get data

In [49]:
# find #typ in text
def find_note(txt, typ, offset):
    '''
    input:
        method to find #typ in text
        txt (string) plain text
        typ (string) type of mark e.g. '#BZ'
        offset (int) starting point for seaerch in text
    output:
        marked text (string)
        offset (integer) starting point for next iterration
    '''
    # find position (index) of #typ in text
    start = txt[offset:].find(typ)
    # if #typ found
    if start > -1:
        start += offset
        # find end of #typ
        if txt[start+1:].find('#') > 0:
            end = txt[start+1:].find('#') + start + 1
            # set offset to end -> starting point for next iterration
            offset = end
            return txt[start:end], offset
        else:
            return typ + " error: missing #", -1
    else:
        return "", -1

In [50]:
# find #typ in text
def find_nutz(txt = '1 LI 1,5 0 160 1 2 2 4 35 Am O Rand'):
    offset = 0
    nutz = []

    for i in range(11):
        end = txt[offset:].find(' ')
        if end != -1:
            nutz.append(txt[offset:offset+end])
            offset += end+1
        else:
            nutz.append('')
    nutz.append(txt[offset:])
    return nutz   

In [ ]:
def get_wo(txt):
    return txt[txt.find('Wirtschaftswald\n')+17:txt.find('\n TO')].strip()

In [51]:
def get_data_page(txt):
    '''
    input:
        search page for all #typ
        txt (string) plain text
    output:
        data (dataframe) containing the data
    '''
    data = []
    # add WO
    data.append(get_wo(txt))
    # loop over all #typ
    for typ in ['#U', '#W', '#STOE', '#VTYP', '#VB', '#UENH', '#UELH', '#BZ', '#PZ', '#ST', '#BE', '#MA',\
                '#N', '#SWE', '#LRV', '#WO']:
        offset = 0
        # loop to find all instances of #N
        if typ == '#N':
            while offset != -1:
                note, offset = find_note(txt, typ, offset)
                note = note[len(typ):]
                nutz = find_nutz(note)
                for n in nutz: 
                    data.append(n)
        else:
            note, offset = find_note(txt, typ, offset)
            data.append(note[len(typ):])
    return data

In [52]:
def get_data_all(pdf_data, start=0, end=1):
    data = []
    for i in range(start, end):
        txt = pdf_data[i]['content']
        data.append(get_data_page(txt))
    # convert to dataframe
    return pd.DataFrame(data)

In [11]:
parsed_data = get_data_all(pdf_data, 0, 100)
parsed_data.to_csv('/home/philipp/Data/obf_autman/csv/dross_01.csv')

In [12]:
#####################################################################################

In [15]:
parsed_data[7] != '1'

0     False
1      True
2     False
3      True
4     False
      ...  
95    False
96    False
97     True
98     True
99     True
Name: 7, Length: 100, dtype: bool

In [24]:
mask = parsed_data[7].isin(['','1','2','3','4']) == False

In [25]:
parsed_data.loc[mask,7]

43    error:
57    error:
77        DE
Name: 7, dtype: object

In [14]:

txt = pdf_data[18]['content']

for typ in ['#BZ', '#N', '#ST', '#BE', '#MA', '#U', '#VTYP']:
    offset=0
    while offset != -1:
        xxx, offset = find_txt(txt, typ, offset)
        print(xxx)

NameError: name 'find_txt' is not defined

In [28]:
pdf_data[11]['content']

'NEU D3\n\nST große Senke, Zugweg in Mitte, stark vergast und verkrautet\nBE aus AD 2015, mit DG/KI aufgeforstet, leider Tannentrieblaus befallen an DG! einige ältere FI/TA aus Vorbestand,\neinzeln LH beigemischt\nMA AF 2015-2018, Rüsselkäferbekämpfung 2016, Schutz gegen Wild, DP Trieblausbefall ne Individuen entnehmen!\n\n8j 88 DG 10 1,0\n    5 KI 6\n    5 FI 7\n    2 TA 8\n\x0c'

In [15]:
txt.count('#BZ')

0

## Get Waldort

In [23]:
txt

' FB 171\n FR 12\n          Forstbetrieb Wienerwald\n          Pernitz\n                                                           Druckdatum: 03.05.2020               BKL       Umtrieb          Wirtschaftswald\n                                                                                                                                                            201 B 3\n TO: 1310 Laufzeit: 01.01.2021- 31.12.2030\n                                                            Seite 4 von 828                   9010          140            in Ertrag\n                                                                                                                                                                       1,98 ha\n    Seeh [m]         Exp           Neig [%]         Neig [°]         STOE                VTYP          Wald-Typ       Wuchs-Geb. Verbissgrad               Schälgrad        fr.SS\n      600            NW              58               30                 26               

In [36]:
start = txt.find('Wirtschaftswald\n')
end = txt.find('TO:')

In [42]:
wo = txt[start+17:end-2]

In [43]:
wo

'                                                                                                                                                           201 B 3'

In [44]:
wo.split()

['201', 'B', '3']

In [45]:
def get_wo(txt):
    start = txt.find('Wirtschaftswald\n')
    end = txt.find('TO:')
    wo = txt[start+17:end-2]
    return wo.split()

In [52]:
txt = pdf_data[11]['content']
get_wo(txt)

['D2',
 'da',
 'zu',
 'Wenig',
 'Platz?',
 '#BZ',
 '4FI',
 '2TA',
 '3BU',
 '#',
 '#STOE',
 '23']

In [ ]:
class OBFPdfPharser(object):

    def __init__(self, data):

        self.data = data

        # fill nan with 0
        self.data = self.data.fillna(0)


    def printit(self):
        print(self.data)
        return(self.data)


    ### 6.1.1 Einschlagsübersicht

    def fuc_tbl_hiebsatzbilanz(self, to, fr, filterx):
        '''
            self.data = input self.data (dataframe)

In [2]:
133 + 55

188

In [3]:
280 + 200

480